## Imports

In [1]:
# General imports
import time
import json
import random

# Intent recognition imports
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# ChatGPT imports
%load_ext autoreload
%autoreload 2
from gpt import GPT
from settings import Settings

# Chat demo
import gradio as gr

## Intent recognition model

In [5]:
# Load the data from JSON file
with open("intent_recognition.json", "r") as f:
    data = json.load(f)

# Define the training data
train_data = []
for datum in data:
    texts = datum["text"]
    label = datum["intent"]
    for text in texts:
        train_data.append((text, label))

# Define the mapping between top-level labels and integers
label_map = {label: i for i, label in enumerate(set([data[1] for data in train_data]))}

# Convert the training data labels to integers using the label_map
labels = torch.tensor([label_map[data[1]] for data in train_data])

# Load the pre-trained BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_map))
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the training data and convert to tensors
inputs = tokenizer.batch_encode_plus([data[0] for data in train_data], padding=True, truncation=True, return_tensors="pt")

# Fine-tune the model on the training data
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4) #default 5e-5
loss_fn = torch.nn.CrossEntropyLoss()
for epoch in range(20):
    optimizer.zero_grad()
    outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    # Evaluate the model on the training data
    predictions = outputs.logits.argmax(axis=1)
    accuracy = (predictions == labels).sum()



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1, Loss: 3.94079327583313
Epoch 2, Loss: 3.9250853061676025
Epoch 3, Loss: 3.8787789344787598
Epoch 4, Loss: 3.8059072494506836
Epoch 5, Loss: 3.736757516860962
Epoch 6, Loss: 3.493802309036255
Epoch 7, Loss: 3.449410915374756
Epoch 8, Loss: 2.9807090759277344
Epoch 9, Loss: 2.923213243484497
Epoch 10, Loss: 2.6897547245025635
Epoch 11, Loss: 2.2915098667144775
Epoch 12, Loss: 2.0994608402252197
Epoch 13, Loss: 1.7259576320648193
Epoch 14, Loss: 1.508161187171936
Epoch 15, Loss: 1.2985793352127075
Epoch 16, Loss: 1.0947285890579224
Epoch 17, Loss: 0.914042055606842
Epoch 18, Loss: 0.7646009922027588
Epoch 19, Loss: 0.6383511424064636
Epoch 20, Loss: 0.5289713144302368


## Save and load saved model

In [6]:
# Save model after training it
# torch.save(model, "intent_recognition.pt")

# Load the model from a file
# model = torch.load("intent_recognition.pt")

## Query intent function

In [8]:
def get_intent(question):

    # Tokenize the test question and convert to tensors
    inputs = tokenizer.encode_plus(question, padding=True, truncation=True, return_tensors="pt")

    # Get the model's prediction for the test question
    with torch.no_grad():
        outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"])
    predictions = outputs.logits.argmax(axis=1)
    predicted_label = list(label_map.keys())[list(label_map.values()).index(predictions[0].item())]
    print(f"Predicted intent: {predicted_label}")
    return predicted_label

# Sample test question
test_question = "Can you summarize the legal statement?"
get_intent(test_question)


Predicted intent: summarization_legal_statement


'summarization_legal_statement'

## ChatGPT model

In [9]:
# GPT model here
# gpt_model = GPT()

gpt_model = False

# Test ChatGPT model
# gpt_model.answer_question(question='What is the most important thing I need to know about your privacy statement?')


# Load Settings class

In [14]:
settings_model = Settings()

user_id = input("Enter the ID of the existing user from the JSON file: ")
settings_model.set_current_user(user_id)
security_level = settings_model.estimate_user_security_level()
print("User security level:", security_level)

User profiles: [{'user_id': '0', 'profile_published': False, 'open_to_connecting': False, 'visibile_in_search': False, 'findable_to': 'logged_on_only', 'individual_user_viewability': [{'user_id': '1', 'profile_card_component_0_visible': True, 'profile_card_component_1_visible': True, 'profile_card_component_2_visible': True, 'profile_card_component_3_visible': True, 'profile_card_component_4_visible': True}, {'user_id': '2', 'profile_card_component_0_visible': True, 'profile_card_component_1_visible': False, 'profile_card_component_2_visible': False, 'profile_card_component_3_visible': False, 'profile_card_component_4_visible': False}]}, {'user_id': '1', 'profile_published': True, 'open_to_connecting': True, 'visibile_in_search': True, 'findable_to': 'everyone'}, {'user_id': '2', 'profile_published': False, 'open_to_connecting': False, 'visibile_in_search': False, 'findable_to': 'logged_on_only'}, {'user_id': '3', 'profile_published': False, 'open_to_connecting': False, 'visibile_in_se

## Get response using intent

In [11]:
def get_object_by_intent(intent):
    for object in data:
        if object['intent'] == intent:
            return object
    return None

# Set up gradio chat for demo

In [12]:
theme = gr.themes.Soft(
    primary_hue="orange",
    secondary_hue="orange",
).set(
    body_text_color_dark='*neutral_800',
    background_fill_primary_dark='*neutral_50',
    background_fill_secondary_dark='*neutral_50',
    border_color_accent_dark='*primary_300',
    border_color_primary_dark='*neutral_200',
    color_accent_soft_dark='*primary_50',
    link_text_color_dark='*secondary_600',
    link_text_color_active_dark='*secondary_600',
    link_text_color_hover_dark='*secondary_700',
    link_text_color_visited_dark='*secondary_500',
    block_background_fill='*neutral_100',
    block_background_fill_dark='*neutral_100',
    block_label_background_fill='*primary_400',
    block_label_background_fill_dark='*primary_400',
    block_label_text_color='*neutral_50',
    block_label_text_color_dark='*neutral_50',
    block_title_text_color='*neutral_50',
    block_title_text_color_dark='*neutral_50',
    checkbox_background_color_dark='*background_fill_primary',
    checkbox_background_color_selected='*primary_500',
    checkbox_background_color_selected_dark='*primary_500',
    checkbox_border_color_dark='*neutral_100',
    checkbox_border_color_focus='*primary_300',
    checkbox_border_color_focus_dark='*primary_300',
    checkbox_border_color_hover_dark='*neutral_300',
    checkbox_border_color_selected='*primary_500',
    checkbox_border_color_selected_dark='*primary_500',
    checkbox_border_width_dark='1px',
    checkbox_label_background_fill_selected_dark='*primary_500',
    checkbox_label_text_color_selected_dark='white',
    error_background_fill_dark='#fee2e2',
    error_border_color_dark='#fecaca',
    input_background_fill_dark='white',
    input_background_fill_focus_dark='*secondary_500',
    input_border_color_dark='*neutral_50',
    input_border_color_focus_dark='*secondary_300',
    input_placeholder_color_dark='*neutral_400',
    slider_color_dark='*primary_500',
    stat_background_fill_dark='*primary_300',
    table_border_color_dark='*neutral_300',
    table_even_background_fill_dark='white',
    table_odd_background_fill_dark='*neutral_50',
    button_primary_background_fill_dark='*primary_500',
    button_primary_background_fill_hover_dark='*primary_400',
    button_primary_border_color_dark='*primary_200',
    button_secondary_background_fill_dark='white',
    button_secondary_background_fill_hover_dark='*neutral_100',
    button_secondary_border_color_dark='*neutral_200',
    button_secondary_text_color_dark='*neutral_800'
)

with gr.Blocks(theme=theme, css="chat/chat.css") as demo:
    gr.Image("https://iyyu.com/_nuxt/img/navbar_logoW@2x.79eba99.png", interactive=False, tool="image", show_label=False, elem_classes="logo").style(width=200)
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    submit = gr.Button("Submit")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]

        intent = get_intent(question=user_message)
        print('intent:', intent)
        
        intent =  get_object_by_intent(intent)
        
        # generating a response with GPT if the main intent was 'privacy_policy' or 'legal_statement'
        use_gpt = intent['use_gpt']

        # if the intent is 'privacy_policy' or 'legal_statement', use GPT to generate a response
        response = '(ChatGPT - intent found: {}): \n{}'.format(intent['intent'], gpt_model.answer_question(question=user_message)) if use_gpt else '(intent found: {}): \n{}'.format(intent['intent'], intent['responses'][0])

        # response = random.choice(response_map[intent])
        history[-1][1] = response
        # The sleep is to simulate a more natural conversation
        time.sleep(1)
        return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    submit.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Predicted intent: update_password_change
intent: update_password_change
Predicted intent: connection_notifications
intent: connection_notifications
